In [ ]:
# Import the lastest version of Tensorflow
import tensorflow

In [ ]:
# Clone the master branch of the Tensorflow Models repository
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Modify the anchor settings here
with open('/content/models/research/object_detection/anchor_generators/multiscale_grid_anchor_generator.py') as f:
    multiscale_grid_anchor_generator = f.read()

with open('/content/models/research/object_detection/anchor_generators/multiscale_grid_anchor_generator.py', 'w') as f: 
  anchor_setting = """scales = [2**(float(scale) / scales_per_octave)
              for scale in range(scales_per_octave)]"""
  multiscale_grid_anchor_generator = multiscale_grid_anchor_generator.replace(anchor_setting, 'scales = [0.8, 1.0, 1.2]')
  f.write(multiscale_grid_anchor_generator)

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
# Test the installation
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
# Clone the nesscessary files from the repo
!git clone https://github.com/Deadfish-hk/AcuCount.git

In [ ]:
# Merge each .xml image labels into one .csv file
!python /content/AcuCount_Dataset/xml_to_csv.py

In [ ]:
# Generate the .tfrecord file for training and testing
%cd /content
!python /content/AcuCount_Dataset/generate_tfrecord.py --csv_input=train_labels.csv --image_dir=/content/AcuCount_Dataset/dataset/train --output_path=train.record
!python /content/AcuCount_Dataset/generate_tfrecord.py --csv_input=test_labels.csv --image_dir=/content/AcuCount_Dataset/dataset/test --output_path=test.record

In [ ]:
# Download the pre-trained model from model zoo
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_1024x1024_coco17_tpu-8.tar.gz
!tar -xf ssd_resnet101_v1_fpn_1024x1024_coco17_tpu-8.tar.gz

In [ ]:
# Insert tfrecords path
train_record_path = '/content/train.record'
test_record_path = '/content/test.record'
labelmap_path = '/content/AcuCount_Dataset/labelmap.pbtxt'

In [ ]:
# Match the checkpoint with your selected model
fine_tune_checkpoint = 'ssd_resnet101_v1_fpn_1024x1024_coco17_tpu-8/checkpoint/ckpt-0'

In [ ]:
# Use your own config file if you want to fine-tune your model. The sample config could be found inside the Tensorflow object detection API repo
base_config_path = '/content/AcuCount_Dataset/research/ssd_resnet101_v1_fpn_1024x1024_coco17_tpu-8.config' 

In [ ]:
# Configure the hyperparameters
batch_size = 1
num_steps = 40000
num_eval_steps = 2000

In [ ]:
import re

with open(base_config_path) as f:
    config = f.read()

with open('model_config.config', 'w') as f:
  
  # Set labelmap path
  config = re.sub('label_map_path: ".*?"', 
             'label_map_path: "{}"'.format(labelmap_path), config)
  
  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)
  
  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
  
  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                  'input_path: "{}"'.format(test_record_path), config)
  
  # Set number of classes. Default set to 1.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(1), config)
  
  # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)
  
  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
  
  # Set fine-tune checkpoint type to detection
  config = re.sub('fine_tune_checkpoint_type: "classification"', 
             'fine_tune_checkpoint_type: "{}"'.format('detection'), config)
  
  f.write(config)

In [ ]:
%cat model_config.config

# SSD with Resnet 101 v1 FPN feature extractor, shared box predictor and focal
# loss (a.k.a Retinanet).
# See Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from Imagenet classification checkpoint
# Train on TPU-8
#
# Achieves 39.5 mAP on COCO17 Val

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 1
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7


In [ ]:
model_dir = 'training/'
pipeline_config_path = '/content/model_config.config'

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/training/train'
%reload_ext tensorboard

<IPython.core.display.Javascript object>

In [ ]:
# Initiate training
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr

2021-12-22 09:28:17.910519: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I1222 09:28:17.914813 139887518218112 mirrored_strategy.py:376] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I1222 09:28:17.920144 139887518218112 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1222 09:28:17.920276 139887518218112 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W1222 09:28:17.952746 139887518218112 deprecation.py:347] From /usr/local/lib/python3.7/dist-packages/object_detection/model_lib_v2.py:564: StrategyBase.experiment

In [ ]:
# Coco Installation (Optional, required when using Coco Evaluation)
%cd /content
!git clone --quiet https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
!cp -r pycocotools /content/models/research/

In [ ]:
# Evaluate the model by COCO detection metrics
%cd /content
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir}  \
    --checkpoint_dir={model_dir} \
    --eval_timeout=600

In [ ]:
# Export the inference graph
# Specify the pathway of where you you want to save the inference graph in
!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir training \
    --output_directory /content/drive/MyDrive/test_infer \
    --pipeline_config_path={pipeline_config_path}